In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

# 1. Giai đoạn EXTRACT
def extract_data_from_db(db_path):
    print("Đang tải dữ liệu từ Database...")
    conn = sqlite3.connect(db_path)
    
    try:
        # Lấy lịch sử bán
        df_sales = pd.read_sql_query("SELECT product_id, all_time_quantity_sold, crawl_timestamp FROM sales_history", conn)
        
        # Lấy lịch sử giá
        df_price = pd.read_sql_query("SELECT product_id, price, crawl_timestamp FROM price_history", conn)
        
        # Lấy đánh giá (review)
        query_rating = """
            SELECT product_id, MAX(review_count) as review_count, MAX(rating_average) as rating_average
            FROM rating_history GROUP BY product_id
        """
        df_rating = pd.read_sql_query(query_rating, conn)
        
        # Lấy thông tin sản phẩm
        df_info = pd.read_sql_query("SELECT id as product_id, name, created_at FROM products", conn)
        
        print(f"   -> Đã load thành công dữ liệu từ {db_path}")
        return df_sales, df_price, df_rating, df_info
    
    except Exception as e:
        print(f"LỖI khi đọc DB: {e}")
        return None, None, None, None
    finally:
        conn.close()


# 2. Giai đoạn TRANSFORM: Xử lý & Tính toán features
def transform_data(df_sales, df_price, df_rating, df_info):
    print("Đang xử lý và tính toán các chỉ số (Feature Engineering)...")
    
    # A. Xử lý thời gian 
    df_sales['crawl_timestamp'] = pd.to_datetime(df_sales['crawl_timestamp'])
    df_price['crawl_timestamp'] = pd.to_datetime(df_price['crawl_timestamp'])
    
    min_date = df_sales['crawl_timestamp'].min()
    max_date = df_sales['crawl_timestamp'].max()
    print(f"   -> Dữ liệu trải dài từ: {min_date} đến {max_date}")

    #  B. Lọc dữ liệu Đầu kỳ & Cuối kỳ 
    
    # 1. Sales (Mới & Cũ)
    df_sales_new = df_sales[df_sales['crawl_timestamp'] == max_date].groupby('product_id')['all_time_quantity_sold'].max().reset_index()
    df_sales_new.rename(columns={'all_time_quantity_sold': 'qty_new'}, inplace=True)
    
    first_day = min_date.date()
    df_sales_old = df_sales[df_sales['crawl_timestamp'].dt.date == first_day].groupby('product_id')['all_time_quantity_sold'].max().reset_index()
    df_sales_old.rename(columns={'all_time_quantity_sold': 'qty_old'}, inplace=True)

    # 2. Price (Mới & Cũ)
    df_price_new = df_price.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='last')[['product_id', 'price']]
    df_price_new.rename(columns={'price': 'price_new'}, inplace=True)
    
    df_price_old = df_price.sort_values('crawl_timestamp').drop_duplicates('product_id', keep='first')[['product_id', 'price']]
    df_price_old.rename(columns={'price': 'price_old'}, inplace=True)

    #  C. Ghép bảng (Merge) 
    # Bắt đầu từ bảng thông tin sản phẩm (đã lọc trùng)
    df_final = df_info.drop_duplicates(subset=['product_id'], keep='last').copy()
    
    # Ghép lần lượt các bảng con vào
    dfs_to_merge = [df_sales_new, df_sales_old, df_price_new, df_price_old, df_rating]
    for df in dfs_to_merge:
        df_final = pd.merge(df_final, df, on='product_id', how='left')

    #  D. Xử lý Missing Value (Imputation) 
    df_final['qty_old'] = df_final['qty_old'].fillna(0)
    df_final['price_old'] = df_final['price_old'].fillna(df_final['price_new'])
    df_final['review_count'] = df_final['review_count'].fillna(0)
    # Nếu sản phẩm bị ẩn/xóa -> Không có số liệu mới -> Coi như không bán được thêm
    df_final['qty_new'] = df_final['qty_new'].fillna(df_final['qty_old'])

    # E. Tính toán chỉ số (Feature Engineering) 
    
    # 1. Target: Quantity Sold in Period
    df_final['quantity_sold_in_period'] = df_final['qty_new'] - df_final['qty_old']

    # 2. Feature: Price Change Rate (%)
    # Tính biến thô (raw) để dùng cho công thức Elasticity
    df_final['price_change_raw'] = (df_final['price_new'] - df_final['price_old']) / df_final['price_old']
    # Tính biến hiển thị (%)
    df_final['price_change_rate'] = (df_final['price_change_raw'] * 100).round(2)

    # 3. Feature: Review Density
    df_final['created_at'] = pd.to_datetime(df_final['created_at'])
    current_time = pd.Timestamp.now()
    
    # 3.1 Tính số ngày tồn tại 
    df_final['days_exist'] = (current_time - df_final['created_at']).dt.days
    # 3.2 Nếu days_exist = 0 (vừa tạo), ta ép nó thành 1. Các số khác giữ nguyên.
    df_final['days_exist'] = df_final['days_exist'].replace(0, 1) 
    
    df_final['review_density'] = (df_final['review_count'] / df_final['days_exist']).round(2)

    # 4. Feature: Price Elasticity
    # Tỷ lệ thay đổi lượng (qty_change_rate)
    qty_change_rate = np.where(
        df_final['qty_old'] > 0,
        df_final['quantity_sold_in_period'] / df_final['qty_old'],
        0
    )
    
    # Elasticity = % Lượng / % Giá
    df_final['price_elasticity'] = np.where(
        (df_final['price_change_raw'] != 0) & (df_final['qty_old'] > 0),
        qty_change_rate / df_final['price_change_raw'],
        0
    )
    df_final['price_elasticity'] = df_final['price_elasticity'].round(2)

    return df_final


# 3. Load
def save_data_to_csv(df, output_dir):
    print("Đang lưu file kết quả...")
    os.makedirs(output_dir, exist_ok=True)

    # 1. File Target Variable
    target_cols = ['product_id', 'name', 'quantity_sold_in_period']
    df[target_cols].to_csv(f'{output_dir}/target_variable.csv', index=False)
    print(f"Đã xuất file 1: target_variable.csv")

    # 2. File Full (Dashboard & Train)
    cols_view = [
        'product_id', 'name', 'created_at',
        'qty_old', 'qty_new', 'quantity_sold_in_period',
        'price_old', 'price_new', 
        'price_change_rate', 'price_elasticity',
        'review_count', 'review_density'
    ]
    df[cols_view].to_csv(f'{output_dir}/feature_analysis_full.csv', index=False)
    print(f"Đã xuất file 2: feature_analysis_full.csv")
    
    print("-" * 30)
    print("Top 10 sản phẩm bán chạy nhất:")
    display(df[cols_view].sort_values(by='quantity_sold_in_period', ascending=False).head(10))


def main():
    DB_PATH = '../data/database/tiki_products_multi.db'
    OUTPUT_DIR = '../data/processed' 
    
    if not os.path.exists(DB_PATH):
        print(f"Không tìm thấy file DB tại: {DB_PATH}")
        return

    # Bước 1: Extract
    sales, price, rating, info = extract_data_from_db(DB_PATH)
    
    if sales is not None: # Nếu load thành công
        # Bước 2: Transform
        final_df = transform_data(sales, price, rating, info)
        
        # Bước 3: Load
        save_data_to_csv(final_df, OUTPUT_DIR)

if __name__ == "__main__":
    main()

Đang tải dữ liệu từ Database...
   -> Đã load thành công dữ liệu từ ../data/database/tiki_products_multi.db
Đang xử lý và tính toán các chỉ số (Feature Engineering)...
   -> Dữ liệu trải dài từ: 2025-11-02 10:45:20.297907 đến 2025-11-23 22:21:23.833893
Đang lưu file kết quả...
Đã xuất file 1: target_variable.csv
Đã xuất file 2: feature_analysis_full.csv
------------------------------
Top 10 sản phẩm bán chạy nhất:


,product_id,name,created_at,qty_old,qty_new,quantity_sold_in_period,price_old,price_new,price_change_rate,price_elasticity,review_count,review_density
79,767101,Bao cao su Durex Kingtex 12 bao,2025-11-23 16:38:18,11851.0,70895.0,59044.0,131000.0,117000.0,-10.69,-46.62,1472,210.29
8,385582,Bao cao su Durex Fetherlite Hộp 12 Bao,2025-11-23 16:38:18,0.0,29837.0,29837.0,203000.0,195000.0,-3.94,0.00,499,71.29
53,543129,Nước Giặt Quần Áo Cho Bé D-nee - Chai 3000ml (...,2025-11-23 16:38:18,206.0,26198.0,25992.0,190000.0,229000.0,20.53,614.70,415,59.29
953,73787185,Sách Những Tù Nhân Của Địa Lý,2025-11-23 16:38:18,0.0,23792.0,23792.0,195000.0,194000.0,-0.51,0.00,4224,603.43
34,505636,Dầu Gội Dưỡng Tóc Siêu Mượt Enchanteur Charmin...,2025-11-23 16:38:18,0.0,18026.0,18026.0,149000.0,149000.0,0.00,0.00,14,2.00
2839,274079594,Kem Đánh Răng Colgate Vitamin C Thơm Mát 170G,2025-11-23 16:38:18,0.0,17244.0,17244.0,28000.0,27000.0,-3.57,0.00,25,3.57
430,24028050,Sữa bột Abbott Pediasure 1.6kg cho trẻ từ 1-10...,2025-11-23 16:38:18,0.0,15122.0,15122.0,1309000.0,1263000.0,-3.51,0.00,824,117.71
439,25108217,Sữa rửa mặt ngăn ngừa mụn Acnes Creamy Wash 100g,2025-11-23 16:38:18,173170.0,186252.0,13082.0,59000.0,59000.0,0.00,0.00,239,34.14
673,53583472,Bộ Hộp Cơm Giữ Nhiệt Lock&Lock Easy Carry 2L L...,2025-11-23 16:38:18,0.0,11934.0,11934.0,1037900.0,1216000.0,17.16,0.00,2718,388.29
41,525590,Sữa Tắm Nước Hoa Enchanteur Sensation 650g,2025-11-23 16:38:18,0.0,10670.0,10670.0,137000.0,137000.0,0.00,0.00,14,2.00
